In [12]:
import string
import re
from os import listdir
from numpy import array
from nltk.corpus import stopwords
from collections import Counter
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
import pydot
import graphviz
import os
path = 'H://practice//txt_sentoken'
os.chdir(path)
os.getcwd()

'H:\\practice\\txt_sentoken'

In [13]:
## load doc into menory
# loaddoc intom memory
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

In [19]:
## turn a doc into tokens
def clean_doc(doc):
    tokens= doc.split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    tokens = [re_punc.sub('', w) for w in tokens]
    ## remove words which are non alphabatic
    tokens = [word for word in tokens if word.isalpha()]
    ## reove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    ## filter out shorter tokens
    tokens = [ word for word in tokens if len(word) >1]
    return tokens

In [20]:
## load doc and add to vocab
def add_doc_to_vocab(filename , vocab):
    ## load doc
    doc = load_doc(filename)
    ## clean doc
    tokens = clean_doc(doc)
    ## update counts
    vocab.update(tokens)

In [21]:
## load all docs in a dictionary
def process_docs(directory, vocab):
    lines = list()
    for filename in listdir(directory):
        if not filename.endswith(".txt"):
            next
        path  = directory + '/' + filename
        add_doc_to_vocab(path, vocab)

In [22]:
## this function works through all documents in a directory  to convert docs into lines

def process_docs(directory, vocab):
# walk through all files in the folder
    for filename in listdir(directory):
# skip any reviews in the test set
        if filename.startswith('cv9'):
           continue
# create the full path of the file to open
        path = directory + '/' + filename
# add doc to vocab
        add_doc_to_vocab(path, vocab)

In [23]:
## save list to file
def save_list(lines, filename):
    ## convert lines to a single blob od text
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [24]:
## define vocab
vocab = Counter()
## add all docs to vocab
process_docs('H:/practice/txt_sentoken/pos', vocab)
process_docs('H:/practice/txt_sentoken/neg', vocab)

## print size of vocab
##print(len(vocab))
##print(vocab.most_common(50))

## keep tokens of a min occurance
min_occr = 2
tokens = [k for k,c in vocab.items() if c >= min_occr]
##print(len(tokens))

## save tokensto a vocab file
save_list(tokens,'vocab1.txt')
## so here I am done with preparing my vocab

In [25]:
## this def will load a doc, clean the text, filter out tokens not in vocab, then return documents as a string of tokens
## load doc, clean and return line of tokens
def doc_to_line(filename, vocab):
    doc = load_doc(filename)
    ## clean doc
    tokens = clean_doc(doc)
    ## filters by vocab
    tokens =[w for w in tokens if w in vocab]
    return ' '.join(tokens)

In [26]:
def process_docs(directory, vocab):
    lines  = list()
# walk through all files in the folder
    for filename in listdir(directory):
        
# skip any reviews in the test set
        if filename.startswith('cv9'):
           continue
# create the full path of the file to open
        path = directory + '/' + filename
        line = doc_to_line(path, vocab)
        lines.append(line)
    return lines

In [27]:
## load and clean a dataset
def load_clean_dataset(vocab):
    ## load documents
    neg = process_docs('H:/practice/txt_sentoken/neg', vocab)
    pos = process_docs('H:/practice/txt_sentoken/pos', vocab)
    docs = neg + pos
    ## prepare labels
    labels = [0 for _ in range(len(neg))] + [1 for _ in range(len(pos))]
    return docs, labels

In [28]:
# load the vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

docs, labels = load_clean_dataset(vocab)
print(len(docs), len(labels))

1800 1800


In [29]:
## prepare process doc for train and test
def process_docs(directory, vocab, is_train):
    lines  = list()
# walk through all files in the folder
    for filename in listdir(directory):
        
# skip any reviews in the test set
        if is_train and filename.startswith('cv9'):
           continue
        if not is_train and not filename.startswith('cv9'):
            continue
# create the full path of the file to open
        path = directory + '/' + filename
        line = doc_to_line(path, vocab)
        lines.append(line)
    return lines

In [30]:
def load_clean_dataset(vocab, is_train ):
    ## load documents
    neg = process_docs('H:/practice/txt_sentoken/neg', vocab, is_train)
    pos = process_docs('H:/practice/txt_sentoken/pos', vocab, is_train)
    docs = neg + pos
    ## prepare labels
    labels = [0 for _ in range(len(neg))] + [1 for _ in range(len(pos))]
    return docs, labels

In [31]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [32]:
## define a model
def define_model(n_words):
    ## define network
    model = Sequential()
    model.add(Dense(50, input_shape=(n_words,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compile network
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize defined model
    model.summary()
    plot_model(model, to_file='model.png', show_shapes=True)
    return model

In [35]:
# load the vocabulary
vocab_filename = 'vocab1.txt'
vocab = load_doc(vocab_filename)
vocab = set(vocab.split())

# load all reviews
train_docs, ytrain = load_clean_dataset(vocab, True)
test_docs, ytest = load_clean_dataset(vocab, False)

In [36]:
# create the tokenizer
tokenizer = create_tokenizer(train_docs)
# encode data
Xtrain = tokenizer.texts_to_matrix(train_docs, mode='freq')
Xtest = tokenizer.texts_to_matrix(test_docs, mode='freq')

In [40]:
# define the model
n_words = Xtest.shape[1]
model = define_model(n_words)
# fit network
model.fit(Xtrain, ytrain, epochs=10, verbose=2)
 #evaluate
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 50)                1288450   
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 51        
Total params: 1,288,501
Trainable params: 1,288,501
Non-trainable params: 0
_________________________________________________________________


ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

In [38]:
import pydot
import graphviz